# Video processing pipeline to concat and accelerate flood videos from Google Cloud Storage bucket

#### Choose base directory

In [1]:
cd ../

C:\Users\luisr\Desktop\Repositories\Data Science Projects\Hackaton COR IV - Centro de Operações do RJ\INCUBAÇÃO\Cameras


#### Define utility functions

In [2]:
import pandas as pd
from IPython.display import clear_output as co
from time import time

# Simples class to report execution time

class Timer:
    def __init__(self):
        self.start = time()
    def end(self, decimals=4):
        end = time() - self.start
        print('\n* TIME TO EXECUTE:', round(end, decimals), 's')
        
# Get blob count, bytes and names from Google Cloud Storage bucket

def gcs_list_folder(folder, ext, bucket_name, print_each=10):
    prefix = folder
    delimiter = None
    names = []
    timer = Timer()
    blobs = gcs.list_blobs(prefix, delimiter, bucket_name)
    for i, blob in enumerate(blobs):
        if blob.name.endswith(ext):
            names.append([blob.name, blob.size])
        if print_each is not None and (i + 1) % print_each == 0:
            print(f'\n- Blobs Searched: {i + 1}'); co(True)
    names = pd.DataFrame(names, columns=['blob_name', 'bytes']) # build blobs dataframe
    print(f'\n * FOLDER: {folder} · EXT: {ext}')
    print(f'\n- Blobs Searched: {i + 1}')
    print(f'\n  · Blobs (Matched): {len(names)}')
    print(f'\n  · Giga Bytes (Matched): {round(names["bytes"].sum() / 1e9, 3)} GB')
    timer.end() # prints time to execute
    return names

---

## Pipeline methods set up

#### Import Google Cloud Storage wrapper module and define storage instance

In [3]:
from modules.googlecloudstorage import GCS

sa_json = '../../../../Apps/APIs/octa-api/credentials/octacity-iduff.json'
user_project = None
default_bucket_name = 'flood-video-collection'

gcs = GCS(sa_json, user_project, default_bucket_name)

#### Write videos with open-cv set up

In [4]:
from modules.video import VideoAnnotator

# Video writer class instance

writer = VideoAnnotator(fps=3, shape=(854, 480), codec='mp4v')

# Accelerated video writer class instance

writer_speed = VideoAnnotator(fps=24, shape=(854, 480), codec='mp4v')

#### Video writer class funcitonality
1. Add running clock to video files
2. Concatenate video files from nested folders
3. Accelerate video files from nested folders

---
# Pipeline Execution

## Step 0 · Pipeline parameters set up 

#### Download from Google Cloud Storage bucket

In [5]:
"""
 - prefix: Folder in any level of the bucket containing sub-folders with videos to feed the pipeline
 * Note: Forward trailing slashses, i.e. `/`, at the end of `prefix` limits results to folders
matching exactly to `prefix`. Otherwise, matches any folder or blob that contains `prefix`.
"""

prefix = 'rivers/manual/'
delimiter = None

bucket_name = 'flood-video-collection' # collection bucket name
folder = 'Dados/flood-video-collection' # bucket collection destination folder

overwrite_download = False

#### Annotate videos timestamps

In [6]:
folder = 'Dados/flood-video-collection'  # local collection source folder
to_folder = 'Dados/flood-video-collection-stamped'  # `time-stamped` local collection source folder
ext = '.mp4' # video file format to search for in nested folders
overwrite_annot = False

#### Concatenate and accelerate videos from folders

In [7]:
base_folder = 'Dados/flood-video-collection-stamped' # `time-stamped` local collection source folder
to_base_folder = 'Dados/flood-video-collection-date' # concatenated local collection destination folder
overwrite_concat = False

#### General purpose parameters

In [8]:
ext = '.mp4'
report_freq = 10

### Step 0.1 · Count blobs and download bytes · ***Preparation Step***

In [9]:
blobs = gcs_list_folder(prefix, ext, bucket_name, print_each=1000)


 * FOLDER: rivers/manual/ · EXT: .mp4

- Blobs Searched: 889

  · Blobs (Matched): 889

  · Giga Bytes (Matched): 0.796 GB

* TIME TO EXECUTE: 1.2863 s


## Step 1 · Download  blobs in Cloud Storage bucket to folder

#### Download blobs in `bucket_name` matching `prefix` to local `folder`

In [10]:
timer = Timer()

gcs.download_to_folder(
    folder, prefix, delimiter, bucket_name,
    overwrite_download, report_freq
)

timer.end()


PREFIX: rivers/manual/ · RUNNING: 1.2 min · RATE: 0.0837 s / file · FINISH-ESTIMATE: 0.0 min · PROGRESS: 880/889 · DOWNLOADS: 60/889

DOWNLOAD FAILED. FILE ALREADY EXISTS. FILE: Dados/flood-video-collection/rivers/manual/3323/rivers/manual/3323/3323/CODE3323 2023-04-07 19-35-03.mp4 · BLOB: rivers/manual/3323/rivers/manual/3323/3323/CODE3323 2023-04-07 19-35-03.mp4 · BUCKET: flood-video-collection · (879/889)

DOWNLOAD FAILED. FILE ALREADY EXISTS. FILE: Dados/flood-video-collection/rivers/manual/3323/rivers/manual/3323/3323/CODE3323 2023-04-07 19-43-06.mp4 · BLOB: rivers/manual/3323/rivers/manual/3323/3323/CODE3323 2023-04-07 19-43-06.mp4 · BUCKET: flood-video-collection · (880/889)

DOWNLOAD FAILED. FILE ALREADY EXISTS. FILE: Dados/flood-video-collection/rivers/manual/3323/rivers/manual/3323/3323/CODE3323 2023-04-07 19-45-03.mp4 · BLOB: rivers/manual/3323/rivers/manual/3323/3323/CODE3323 2023-04-07 19-45-03.mp4 · BUCKET: flood-video-collection · (881/889)

DOWNLOAD FAILED. FILE ALREAD

## Step 2 · Annotate videos with dinamic timestamps

#### Add clock timestamp to nested video files in `folder`

In [14]:
timer = Timer()

writer.write_timestamp_to_nested_videos(folder, to_folder, ext, overwrite_annot, report_freq=50)

timer.end()

VIDEO TIMESTAMP ANNOTATION · DONE: 15450/15470 · SUCCESS: 0/15470
ANNOTATE VIDEO TIMESTAMP FAILED. FILE ALREADY EXISTS · FILE: Dados/flood-video-collection-stamped/rivers/manual/3323/CODE3323 2023-05-10 06-45-00.mp4
ANNOTATE VIDEO TIMESTAMP FAILED. FILE ALREADY EXISTS · FILE: Dados/flood-video-collection-stamped/rivers/manual/3323/CODE3323 2023-05-10 07-00-01.mp4
ANNOTATE VIDEO TIMESTAMP FAILED. FILE ALREADY EXISTS · FILE: Dados/flood-video-collection-stamped/rivers/manual/3323/CODE3323 2023-05-10 07-15-00.mp4
ANNOTATE VIDEO TIMESTAMP FAILED. FILE ALREADY EXISTS · FILE: Dados/flood-video-collection-stamped/rivers/manual/3323/CODE3323 2023-05-10 07-30-00.mp4
ANNOTATE VIDEO TIMESTAMP FAILED. FILE ALREADY EXISTS · FILE: Dados/flood-video-collection-stamped/rivers/manual/3323/CODE3323 2023-05-10 07-45-00.mp4
ANNOTATE VIDEO TIMESTAMP FAILED. FILE ALREADY EXISTS · FILE: Dados/flood-video-collection-stamped/rivers/manual/3323/CODE3323 2023-05-10 08-00-01.mp4
ANNOTATE VIDEO TIMESTAMP FAILED. F

## Step 3 · Concatenate and accelerate videos from folders

#### Concatenate videos by date from nested folders in `base_folder`

In [16]:
timer = Timer()

writer_speed.concatenate_videos_by_date_from_nested_folders(
    base_folder, to_base_folder, ext, overwrite_concat, report_freq=5
)

timer.end()

CONCAT VIDEOS BY DATE FROM NESTED FOLDERS · DONE: 430/432 · FOLDER: rivers/manual/1487
CONCAT VIDEOS BY DATE FROM FOLDER (SUCCESS) · FILE-CREATED:  CODE3318 2023-05-09.mp4
CONCAT VIDEOS BY DATE FROM FOLDER (SUCCESS) · FILE-CREATED:  CODE3318 2023-05-10.mp4
CONCAT VIDEOS BY DATE FROM FOLDER (SUCCESS) · FILE-CREATED:  CODE3323 2023-04-07.mp4
CONCAT VIDEOS BY DATE FROM FOLDER (SUCCESS) · FILE-CREATED:  CODE3323 2023-04-08.mp4
CONCAT VIDEOS BY DATE FROM FOLDER (SUCCESS) · FILE-CREATED:  CODE3323 2023-05-09.mp4
CONCAT VIDEOS BY DATE FROM FOLDER (SUCCESS) · FILE-CREATED:  CODE3323 2023-05-10.mp4

* TIME TO EXECUTE: 236.3582 s
